In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from imblearn.under_sampling import RandomUnderSampler
import re

In [2]:
df=pickle.load(open('stem_data.pkl','rb'))
df=pd.DataFrame(df)
df.head()

,text,label
0,washington reuter head conserv republican fact...,1
1,washington reuter transgend peopl allow first ...,1
2,washington reuter special counsel investig lin...,1
3,washington reuter trump campaign advis georg p...,1
4,seattl washington reuter presid donald trump c...,1


In [22]:
x=df[['text']]
y=df.iloc[:,-1]

In [23]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [24]:
rus = RandomUnderSampler(random_state=2, sampling_strategy = 'majority')

In [25]:
x_res,y_res = rus.fit_resample(x,y)

In [26]:
x_res=x_res['text']
x_res

445      donald trump drag us war north korea could wit...
778      total noth els better trump administr crack ma...
892      alleg presid donald trump announc twitter time...
896      get readi donald trump throw worst temper tant...
974      white hous counselor kellyann conway like call...
                               ...                        
14056    brussel reuter british prime minist theresa ma...
21085    xiamen china reuter nation bric group strongli...
8380     washington reuter u presid barack obama still ...
14114    geneva reuter syrian besieg enclav eastern gho...
7373     washington reuter u senat major leader mitch m...
Name: text, Length: 4540, dtype: object

In [27]:
x_train,x_test,y_train,y_test = train_test_split(x_res,y_res,test_size=0.2,random_state=2)

In [28]:
port_stem = PorterStemmer()
vect=TfidfVectorizer()
lr = LogisticRegression()
x_train_vector = vect.fit_transform(x_train).toarray()
x_test_vector = vect.transform(x_test).toarray()
x_train_vector

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
lr.fit(x_train_vector,y_train)

LogisticRegression()

In [31]:
def Fake_news(news):
    te = re.sub('[^a-zA-Z]',' ',news)
    te = te.lower()
    te = te.split()
    te = [port_stem.stem(word) for word in te if not word in stopwords.words('english')]
    te = ' '.join(te)
    te=[te]
    te = vect.transform(te).toarray()
    te=lr.predict(te)
    return te

In [39]:
true=pd.read_csv('True.csv')
fake=pd.read_csv('Fake.csv')

In [67]:
true=true[['text']]
news=true.iloc[20000].values[0]

In [76]:
fake=fake[['text']]
news=fake.iloc[2000].values[0]

In [74]:
Fake_news(news)

array([0], dtype=int64)

In [48]:
lr.score(x_test_vector,y_test)*100

97.13656387665198

In [77]:
news

'No matter what Donald Trump does or where he goes, he s never actually doing the right thing and fulfilling his duties as POTUS.One might think that after suffering such a monstrous fail with his American Health Care Act, the president might double down and get right back to work on his quest to somehow  improve  Obamacare, which he insists is awful despite the fact the Americans overwhelmingly approve of it. But no   instead, Trump decided to spend his Saturday the way he s spent pretty much every Saturday since he became president   by playing golf.In the short 9 weeks of his presidency, Trump has already gone golfing 12 times   which is far more than any of his predecessors and former President Barack Obama, whom Trump once criticized for taking any downtime to play golf. Trump has been getting blasted for his weekend golfing getaways, and the White House has gone to great lengths to hide it:However, Trump s cover was blown when some Instagram photographs of Trump surfaced, reveali

In [58]:
pickle.dump(lr,open('lr.pkl','wb'))

In [60]:
l2=pickle.load(open('lr.pkl','rb'))

In [63]:
pickle.dump(vect,open('vect.pkl','wb'))